In [1]:
import os
import pandas as pd
import numpy as np
import time
import gc
import pickle

In [2]:
from process import assign_atoms_index, get_pickup_dist_matrix

In [3]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'

In [ ]:
# df_mulliken_charges = pd.read_csv(FOLDER + 'mulliken_charges.csv')
# df_sample =  pd.read_csv(FOLDER + 'sample_submission.csv')
# df_magnetic_shielding_tensors = pd.read_csv(FOLDER + 'magnetic_shielding_tensors.csv')
df_train = pd.read_csv(FOLDER + 'train.csv')
# df_test = pd.read_csv(FOLDER + 'test.csv')
# df_dipole_moments = pd.read_csv(FOLDER + 'dipole_moments.csv')
# df_potential_energy = pd.read_csv(FOLDER + 'potential_energy.csv')
df_structures = pd.read_csv(FOLDER + 'structures.csv')
# df_scalar_coupling_contributions = pd.read_csv(FOLDER + 'scalar_coupling_contributions.csv')

In [ ]:
df_structures_idx = df_structures.set_index('molecule_name')
df_train_idx = df_train.set_index('molecule_name')

In [ ]:
mols = df_train['molecule_name'].unique()

In [ ]:
num = 7
dist_mat = np.zeros([0, num*5*4])
num_div = len(mols) // 4
atoms_idx = np.zeros([0], dtype=np.int32)
molecule_names = np.empty([0])

start = time.time()

for mol in mols[:num_div]:
    assigned_idxs = assign_atoms_index(df_train_idx, mol)
    dist_mat_mole = get_pickup_dist_matrix(df_train_idx, df_structures_idx, mol, num_pickup=num)
    mol_name_arr = [mol] * len(assigned_idxs) 
    
    molecule_names = np.hstack([molecule_names, mol_name_arr])
    atoms_idx = np.hstack([atoms_idx, assigned_idxs])
    dist_mat = np.vstack([dist_mat, dist_mat_mole])
    
col_name_list = []
atoms = ['H', 'C', 'N', 'O', 'F']
for a in atoms:
    for n in range(num):
        col_name_list.append('dist_{}_{}'.format(a, n))
    for n in range(num):
        col_name_list.append('bond_cos2_{}_{}'.format(a, n))
    for n in range(num):
        col_name_list.append('bond_cos3_{}_{}'.format(a, n))
    for n in range(num):
        col_name_list.append('cos_{}_{}'.format(a, n))
        
se_mole = pd.Series(molecule_names, name='molecule_name')
se_atom_idx = pd.Series(atoms_idx, name='atom_index').astype('int32')
df_dist = pd.DataFrame(dist_mat, columns=col_name_list).astype('float32')
df_distance = pd.concat([se_mole, se_atom_idx,df_dist], axis=1)

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

In [ ]:
for i in range(num):
    del df_distance['bond_cos2_H_{}'.format(i)]
    del df_distance['bond_cos2_O_{}'.format(i)]
    del df_distance['bond_cos2_F_{}'.format(i)]
    del df_distance['bond_cos3_H_{}'.format(i)]
    del df_distance['bond_cos3_O_{}'.format(i)]
    del df_distance['bond_cos3_F_{}'.format(i)]

In [ ]:
nums = [num, num, 5, 4, 3]
for a, n in zip(atoms, nums):
    for i in range(n, num):
        del df_distance['dist_{}_{}'.format(a, i)]
        del df_distance['cos_{}_{}'.format(a, i)]

In [ ]:
df_distance.to_pickle(OUTPUT + '20190701_dist_bond_dir_train_temp0.pickle')

In [ ]:
df_distance.head()